<i>Le programme contenu dans ce Notebook permet de jouer au jeu de l'Hexapion. (deux joueurs humains s'affrontent ou un joueur humain affronte l'ordinateur qui choisit ses coups au hasard.</i>

<span style="color: #9317B4"> Pour exécuter une saisie Python, sélectionner la cellule et valider avec </span><span style="color: #B317B4"><strong>SHIFT+Entrée</strong></span>.


In [ ]:
"""
Jeu de l'Hexapion 
Ce programme permet à deux joueurs humains de s'affronter ou à un joueur humain d'affronter un bot qui joue aléatoirement ses coups
Les variables globales N et M correspondent aux dimensions du plateau de jeu (largeur N=3 et hauteur M=3 par défaut)
"""

N=3 #N :largeur du plateau de jeu
M=3 #M :longueur du plateau de jeu

# 0 1 ... N (<-ligne des noirs)
# 1
# .
# .
# M-1 ...   (<-ligne des blancs) 

from random import choice
from copy import deepcopy

class position:
    
    def __init__(self,grille=[ [False if j==0 else True if j==M-1 else None for i in range(N)] for j in range(M)],tour_de_jeu=True):
        """
        Création d'une position.
        Par défaut, position de départ avec tour de jeu aux blancs
        (conventions : True = blanc, False = noir)
        """
        
        self.grille = grille 
        
        self.tour_de_jeu = tour_de_jeu
        
    def __eq__(self,other): #symbole ==
        """
        indique si deux positions sont identiques
        """
        return self.tour_de_jeu == other.tour_de_jeu and self.grille == other.grille
        
    def __invert__(self): #symbole ~ devant la position    
        """
        renvoie la position symétrique (avec même tour de jeu)
        """
        sym = position()
        return position([ ligne[::-1] for ligne in self.grille ],self.tour_de_jeu)
        
    def coups(self):
        """
        renvoie la liste des positions suivantes possibles
        """
        liste_coups=[]
        
        #on parcourt toutes les cases de la grille
        for m in range(len(self.grille)):
            for n in range(len(self.grille[m])):
                
                #variable pour le sens de mouvement
                sens = -1 if self.tour_de_jeu else 1
                #on repère les pions de la couleur qui joue
                
                if self.grille[m][n] == self.tour_de_jeu:

                    #on teste si le pion peut prendre ou avancer (trois directions possibles)
                    for dir in [-1,0,1]:
                        
                        #test enlevé0 <= m+sens <M and
                        
                        if 0<= n+dir <N and self.grille[m+sens][n+dir] == (None if dir==0 else not self.tour_de_jeu):
                            coup=position(deepcopy(self.grille),not self.tour_de_jeu) #on duplique la position (avec changement de tour de jeu)
                            coup.grille[m][n] = None #on vide la case du pion déplacé
                            coup.grille[m+sens][n+dir] = self.tour_de_jeu #on place le pion dans sa nouvelle position
                            liste_coups.append(coup) #on ajoute le coup à la liste des coups possibles
                            

                            
        return liste_coups
        
    def gain(self):
        """
        renvoie le joueur gagnant:
        True  -> blanc
        False -> noir
        None  -> pas encore de vainqueur
        """
        
        #si un pion a atteint la dernière rangée, il y a un vainqueur
        if False in self.grille[M-1]: return False
        if True in self.grille[0]: return True
        
        #s'il n'y a plus de coups possibles, il y a un vainqueur
        #(NB: ce test gère aussi a fortiori le cas où il n'y a plus de jeton à jouer)
        if self.coups()==[]: return not self.tour_de_jeu
        
        #sinon il n'y a pas encore de vainqueur
        return None
            
    def genere_lignes_affichage(self):
        """
        génère une liste contenant chaque ligne d'affichage de la position dans la console
        "O" = pion blanc 
        " " = case vide
        "X" = pion noir
        ─┐┌└┘│
        
        Pour permettre une optimisation de l'affichage si on veut juxtaposer des grilles, 
        la méthode renvoie la liste des lignes (lignes sous forme de str)
        N :largeur du plateau de jeu
        M :longueur du plateau de jeu
        """
        Aff_grille = ["┌"+"─"*N+"┐"]
        for ligne in self.grille:
            Aff_ligne = "│"
            for case in ligne:
                Aff_ligne += " " if case is None else "O" if case else "X"            
            Aff_ligne += "│"
            Aff_grille.append(Aff_ligne)
                
            
        Aff_grille.append("└"+"─"*N+"┘")
        Aff_grille.append("Tour:"+" "*(N-3)) 
        Aff_grille.append(("Blanc" if self.tour_de_jeu else "Noir ")+" "*(N-3))
        return Aff_grille  
        
    def __str__(self):
        """
        Conversion en chaîne de caractère
        Permet d'utiliser la syntaxe print pour un objet position
        """    
        Aff=""
        for ligne in self.genere_lignes_affichage(): Aff += ligne+"\n"
        return Aff

def partie(ordi=None):
    """
    Fonction qui permet de s'affronter au jeu de l'Hexapion
    ordi -> None : affrontement de deux joueurs humains
         -> True : l'ordinateur joue les blancs (coups au hasard)
         -> False: l'ordinateur joue les noirs  (coups au hasard)

    """
    
    print("┌────────────────────────────────────────────────┐ ")
    print("│Jeu de l'Hexapion - Démarrage de partie         │ ")   
    print("│                                                │ ")
    print("│Blanc :"+ ("ordi  " if ordi==True  else "humain")+" "*35+"│")
    print("│Noir  :"+ ("ordi  " if ordi==False else "humain")+" "*35+"│")
    print("└────────────────────────────────────────────────┘\n")
    
    pos = position()
        
    # Affichage de la position courante
    print("\n_________________________________\nPosition actuelle:\n")
    print(pos)
        
    
    # Tant que personne n'a gagné
    while pos.gain() is None:
        
        # Calcul des coups suivants possibles
        coups_possibles = pos.coups()

        if pos.tour_de_jeu==ordi:
            #Si c'est l'ordinateur qui doit jouer
            pos = choice(coups_possibles)            
        else:
            #Sinon (c'est l'humain qui joue)
        
            # Message requête
            print("Choisir un coup:")
            
            # Affichage des n° des coups possibles
            
            Affichage= [ coups_possibles[k].genere_lignes_affichage() for k in range(len(coups_possibles)) ]
                    
            ligne=""
            for j in range(len(coups_possibles)):
                ligne += str(j)+" "*(N+4)
            print(ligne)
            
            # Affichage des coups possibles
            
            for j in range(len(Affichage[0])):
                ligne=""
                for k in range(len(Affichage)): ligne += Affichage[k][j]+"   " 
                print(ligne)
            
            # Attente d'un choix de coup valide
            
            while True:
                try:
                    choix = input("\nChoix:")
                    pos = coups_possibles[int(choix)]
                    break
                except:
                    print("Saisie invalide.")
        
        # Affichage de la position courante
        print("\n_________________________________\nPosition actuelle:\n")
        print(pos)
        
    # Renvoie l'indication du joueur gagnant    
    return "!!! Gain blanc !!!" if pos.gain() else "!!! Gain noir !!!"
    
    
print("""
Exécuter la fonction partie pour démarrer.
partie() permet à deux joueurs humains de s'affronter.
partie(True) permet d'affronter l'ordinateur qui joue les blancs.
partie(False) permet d'affronter l'ordinateur qui joue les noirs.
""")


In [ ]:
# Exécuter cette cellule permet à deux joueurs de s'affronter
partie()

In [ ]:
# Exécuter cette cellule permet de jouer les blancs contre l'ordinateur
partie(False)

In [ ]:
# Exécuter cette cellule permet de jouer les noirs contre l'ordinateur
partie(True)